# Jeu de Taquin
_following Florent Hivert's https://github.com/hivert/SageWidgetExper_

In [ ]:
from jeudetaquin import JeuDeTaquin
from sage_widget_adapters.combinat.skew_tableau_grid_view_adapter import SkewTableauGridViewAdapter
class JeuDeTaquinGridViewAdapter(SkewTableauGridViewAdapter):
    objclass = JeuDeTaquin
    constructorname = 'JeuDeTaquin'
    addablecelltype = bool
    addablecellzero = False

    # Convert widget value to Sage cell value
    def display_to_cell(self, display_value, display_type):
        if type(display_value) == bool: # Case of a hole
            return display_value
        return super(JeuDeTaquinGridViewAdapter, self).display_to_cell(display_value, display_type)

    # Compute clickable cells coordinates
    @staticmethod
    def addable_cells(obj):
        if obj._hole:
            return [obj._hole] # Return the hole
        return obj.inner_shape().corners() # Return all potential holes
    
    # What happens when you click
    @classmethod
    def add_cell(cls, obj, pos, val, dirty={}):
        # Create a hole if there isn't
        if not obj.has_hole():
            obj.create_hole(pos)
        # Slide
        obj.slide()
        return obj

    # Compute widget cells
    @staticmethod
    def compute_cells(obj):
        cells = SkewTableauGridViewAdapter.compute_cells(obj)
        if obj._hole:
            cells[obj._hole] = False
        if obj._new_st:
            for i in range(len(obj._new_st)):
                for j in range(len(obj._new_st[i])):
                    if obj._new_st[i][j] > 0:
                        cells[(i,j)] = obj._new_st[i][j]
                    elif (i,j) in cells:
                        del(cells[(i,j)])
        return cells

#t = SkewTableau([[None,2,3],[None,4],[5]])
t = SkewTableau([[None,None,None,4,4,5,6,7],[None,2,4,6,7,7,7],[None,4,5,8,8,9],[None,6,7,10],[None,8,8,11],[None],[4]])
jdt = JeuDeTaquin(t)
A = JeuDeTaquinGridViewAdapter()

In [ ]:
from sage_combinat_widgets.grid_view_widget import DisabledTextCell, AddableButtonCell, GridViewWidget
from ipywidgets import Layout
hole_layout = Layout(width='3em',height='2em', margin='0', padding='20px')

# Class for clickable cells
class JDTHoleWidget(AddableButtonCell):
    def __init__(self, position, layout=None, **kws):
        super(JDTHoleWidget, self).__init__(position, layout, **kws)
        self.description = ''
        self.tooltip = 'Click here to play!'
        self.add_class('hole')

# JeuDeTaquin as a widget
w = GridViewWidget(jdt, A, display_convention='fr', cell_widget_classes=[DisabledTextCell], addable_widget_class=JDTHoleWidget)

In [ ]:
%%html
<style>.hole {background-color: turquoise ; margin:1px !important}</style>

In [ ]:
# The player sees the initial jeu de taquin, with an clickable button for each possible hole.
w

In [ ]:
w.addable_cells()

In [ ]:
w.value._hole